In [9]:
import tensorflow as tf
import numpy as np

def AddGCNLayer(feature, adj_matrix, d_matrix, in_size, out_size, activation=None,):
    Weights = tf.Variable(tf.random_normal([out_size, in_size]))
    Propa = tf.matmul(Weights, tf.matmul(d_matrix,tf.matmul(adj_matrix,feature)))
    if activation is None:
        outputs = Propa
    else:
        outputs = activation(Propa)
    return outputs

def AddDenseLayer(inputs, in_size, out_size, activation=None,):
    Weights = tf.Variable(tf.random_normal([out_size, in_size]))
#     biases = tf.Variable(tf.zeros([out_size, 1]) + 0.01,)
    Wx_plus_b = tf.matmul(Weights, inputs)
    if activation is None:
        outputs = Wx_plus_b
    else:
        outputs = activation(Wx_plus_b,axis=0)
    return outputs

fea_matrix = np.load('fea_matrix.npy',allow_pickle=True)
w_adj_matrix = np.load('w_adj_matrix.npy',allow_pickle=True)
label = np.load('label.npy',allow_pickle = True).reshape([len(fea_matrix),2,1])
adj_matrix = np.load('adj_matrix.npy',allow_pickle=True)
d_matrix = np.load('d_matrix.npy',allow_pickle=True)

fea = tf.placeholder(tf.float32,[14,1])
w_adj = tf.placeholder(tf.float32,[14,14])
d = tf.placeholder(tf.float32,[14,14])
la = tf.placeholder(tf.float32,[2,1])


x1 = AddGCNLayer(fea,w_adj,d,14,8,activation=tf.nn.sigmoid)
x2 = AddDenseLayer(x1,8,2,activation=tf.nn.softmax)


# loss = tf.losses.softmax_cross_entropy(onehot_labels=la,logits=x2)
loss = tf.reduce_mean(-tf.reduce_sum(la * tf.log(x2), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(0.01).minimize(loss)

initi = tf.global_variables_initializer()
sess = tf.Session()
sess.run(initi)

test_raw = []
for i in range(10):
    test_raw.append([])
for i in range(100000):
    sess.run(train_step,feed_dict={fea:fea_matrix[i],w_adj:adj_matrix[i],d:d_matrix[i],la:label[i]})
    if i%10000 == 0:
        for j in range(int(len(fea_matrix)-100000)):
            test_raw[int(i/10000)].append(sess.run(x2,feed_dict={fea:fea_matrix[j+100000],
                                                                    w_adj:adj_matrix[j+100000],
                                                                    d:d_matrix[j+100000],
                                                                    la:label[j+100000]}))
        print(i,'/100000 finished!')

test = tf.placeholder(tf.float32,[None,None,2,1])
predict = sess.run(tf.round(test),feed_dict={test:test_raw})

acc = []
for i in range(10):
    acc.append([])
for i in range(10):
    for j in range(len(predict[0])):
        if bool((predict[i][j]==label[100000:][j])[0]):
            acc[i].append(True)
        else:
            acc[i].append(False)

for i in range(10):
    print(i,acc[i].count(True)/len(acc[i]))

0 /100000 finished!
10000 /100000 finished!
20000 /100000 finished!
30000 /100000 finished!
40000 /100000 finished!
50000 /100000 finished!
60000 /100000 finished!
70000 /100000 finished!
80000 /100000 finished!
90000 /100000 finished!
0 0.4731905616242364
1 0.5268094383757637
2 0.5143869370699248
3 0.5339404537505962
4 0.5360979265550837
5 0.5268094383757637
6 0.5312152249449277
7 0.5355074603138555
8 0.537846615038721
9 0.5300797129425658
